In [36]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pathlib
from tensorflow.keras import layers
data_root_orig = tf.keras.utils.get_file(origin='https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz',
                                         fname='notMNIST_small', untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

/Users/max/.keras/datasets/notMNIST_small


In [15]:
for letter_dir in data_root.iterdir():
  print(letter_dir)

/Users/max/.keras/datasets/notMNIST_small/I
/Users/max/.keras/datasets/notMNIST_small/G
/Users/max/.keras/datasets/notMNIST_small/A
/Users/max/.keras/datasets/notMNIST_small/F
/Users/max/.keras/datasets/notMNIST_small/H
/Users/max/.keras/datasets/notMNIST_small/J
/Users/max/.keras/datasets/notMNIST_small/C
/Users/max/.keras/datasets/notMNIST_small/D
/Users/max/.keras/datasets/notMNIST_small/E
/Users/max/.keras/datasets/notMNIST_small/B


In [17]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

18724

In [18]:
all_image_paths[:10]

import IPython.display as display

for n in range(5):
  image_path = random.choice(all_image_paths)
  display.display(display.Image(image_path))
  print(image_path)

/Users/max/.keras/datasets/notMNIST_small/J/QmVhdSBUaGluIEl0YWxpYy50dGY=.png


/Users/max/.keras/datasets/notMNIST_small/I/QmFzZU1vbm9XaWRlVGhpbiBSZWd1bGFyLnR0Zg==.png


/Users/max/.keras/datasets/notMNIST_small/H/Q3Jvc3NvdmVyIEV4dHJhTGlnaHQudHRm.png


/Users/max/.keras/datasets/notMNIST_small/J/SG91c2VHb3RoaWNIRzIzVGV4dC1MaWdodC5vdGY=.png


/Users/max/.keras/datasets/notMNIST_small/F/Q2FlY2lsaWEtSGVhdnlJdGFsaWNPc0Yub3Rm.png


In [19]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [20]:
label_to_index = dict((name, index) for index, name in enumerate(label_names))
label_to_index

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9}

In [21]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])
print("First 10 paths: ", all_image_paths[:10])

First 10 labels indices:  [8, 5, 6, 6, 3, 3, 0, 3, 7, 1]
First 10 paths:  ['/Users/max/.keras/datasets/notMNIST_small/I/Q2hhbGV0Qm9vayBCb2xkLm90Zg==.png', '/Users/max/.keras/datasets/notMNIST_small/F/Q2VudHVyeTcyNUJULUl0YWxpYy5vdGY=.png', '/Users/max/.keras/datasets/notMNIST_small/G/QWxwaGEgRmxpZ2h0IFNvbGlkIFNtYWxsIENhcHMudHRm.png', '/Users/max/.keras/datasets/notMNIST_small/G/Q2FzYWJsYW4tRXh0cmFCb2xkLm90Zg==.png', '/Users/max/.keras/datasets/notMNIST_small/D/SVRDIFRpZmZhbnkgTWVkaXVtLnBmYg==.png', '/Users/max/.keras/datasets/notMNIST_small/D/R2VvbWV0cmljIDg4NS5wZmI=.png', '/Users/max/.keras/datasets/notMNIST_small/A/Q3Jvbm9zUHJvLUJvbGRTdWJoLm90Zg==.png', '/Users/max/.keras/datasets/notMNIST_small/D/R2FyYW1vbmRIYW5kdG9vbGVkSVRDU3RkLm90Zg==.png', '/Users/max/.keras/datasets/notMNIST_small/H/Q29tc2F0LU5hdnktVW5pdC5vdGY=.png', '/Users/max/.keras/datasets/notMNIST_small/B/SG91c2VHb3RoaWNIRzIzVGV4dC1MaWdodC5vdGY=.png']


In [66]:
import numpy as np

VALIDATION_SET_RATE = 0.2

# define image generators for train, validation and test set
image_generator_train = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=VALIDATION_SET_RATE
)

BATCH_SIZE = 32
IMG_HEIGHT = 28
IMG_WIDTH = 28
IMG_CHANNELS = 3



train_data_gen = image_generator.flow_from_directory(directory=str("/Users/max/.keras/datasets/notMNIST_small/"),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     subset='training')
STEPS_PER_EPOCH_TRAIN = np.ceil(train_data_gen.samples/BATCH_SIZE)

validation_data_gen = image_generator.flow_from_directory(directory=str("/Users/max/.keras/datasets/notMNIST_small/"),
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     subset='validation')

STEPS_PER_EPOCH_VALIDATION = np.ceil(validation_data_gen.samples/BATCH_SIZE)

Found 18724 images belonging to 10 classes.
Found 0 images belonging to 10 classes.


In [61]:
from tensorflow import keras

model = keras.Sequential([
        layers.Conv2D(16, kernel_size=(3, 3), padding='same', input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.1),
        layers.Dense(64, activation='relu'),
        layers.Dense(10, activation='softmax', name='output')
    ])
model.summary()
    
# Compile model
model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
model.fit_generator(
    train_data_gen,
    validation_data=validation_data_gen,
    steps_per_epoch=STEPS_PER_EPOCH_TRAIN,
    validation_steps=STEPS_PER_EPOCH_VALIDATION,
    epochs=3)

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 28, 28, 16)        448       
_________________________________________________________________
flatten_11 (Flatten)         (None, 12544)             0         
_________________________________________________________________
dense_56 (Dense)             (None, 64)                802880    
_________________________________________________________________
dropout_48 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_57 (Dense)             (None, 64)                4160      
_________________________________________________________________
output (Dense)               (None, 10)                650       
Total params: 808,138
Trainable params: 808,138
Non-trainable params: 0
_______________________________________________